In [1]:
import ipywidgets as ipw
import json
import random
import time
import pandas as pd
import os
import webbrowser
import math
from IPython.display import display, Markdown

Copyright **Jacob Martin and Paolo Raiteri**, January 2021

## Numerical solution of chemical equilibrium problems #3
This notebook solves the equilibrium problem of a monoprotic acid in an external buffer.
Assume that this is the only reaction taking plane in the system.

\begin{equation}
HA \rightleftharpoons A^{-} + H^+
\end{equation}


- Click `Download CSV` to export the data as a CSV file to verify your result.


In [2]:
def initialise():
    global nPoints
    global concA, concB, pH
    global pKeq1
    global delta

    nPoints = 100
    concA = 0.1
    concB = 0.1
    pH = 6
    pKeq1 = 3.74
    delta = 0.1

def addLine(t,x,y,q1,res):
    var_list = []
    var_list.append(t)
    var_list.append(x)
    var_list.append(y)
    var_list.append(q1)
    res.loc[len(res)] = var_list

initialise()


In [4]:
respath = os.path.join(os.getcwd(), "..", "results.csv")

out_P = ipw.Output()
out_L = ipw.Output()

with out_L:
    display(Markdown("[Download CSV](../results.csv)"))

    
def force(Q,k):
    if (abs(Q) > 1.e-6):
        force = - math.log(Q/k)
    else:
        force = 1.

    return force

def calc1(btn):
    out_P.clear_output()
    
    if os.path.exists(respath):
        os.remove(respath)
    res = pd.DataFrame(columns=["step" 
                                , "CH$_3$COOH" 
                                , "CH$_3$COO$^{-}$" 
                                , "$Q/K_{eq}$"
                               ])

    A  = float(concA_text.value)
    B  = float(concB_text.value)
    H  = math.pow(10,-float(pH_text.value))

    dx = float(delta_text.value)
    pk1 = float(pKeq1_text.value)
    k1 = math.pow(10,-pk1)
    n  = int(nPoints_text.value)

    Q1 = B * H / A
    
    addLine(1,A,B,
            Q1/k1,
            res)
    
    for i in range(0, n):
        f1 = force(Q1,k1)

        dx1 = min(A,B) * dx * f1
        
        A = A - dx1
        B = B + dx1

        Q1 = B * H / A            

        addLine(i+1,A,B,
                Q1/k1,
                res)

    res.to_csv(respath, index=False)
    with out_P:
        display(res.tail(n))

btn_calc1 = ipw.Button(description="Get Data", layout=ipw.Layout(width="150px"))
btn_calc1.on_click(calc1)

rows = []

# Equilibrium constant
pKeq1_text = ipw.Text(str(pKeq1))

# Initial concentrations
concA_text = ipw.Text(str(concA))
concB_text = ipw.Text(str(concB))
pH_text = ipw.Text(str(pH))

# delta concentration
delta_text = ipw.Text(str(delta))

# Nmber of data points
nPoints_text = ipw.Text(str(nPoints))

label_layout = ipw.Layout(width='300px')


rows.append(ipw.HBox([ipw.Label('Initial concentration of "HA"      ',layout=label_layout),concA_text]))
rows.append(ipw.HBox([ipw.Label('Initial concentration of "A$^{-}$" ',layout=label_layout),concB_text]))
rows.append(ipw.HBox([ipw.Label('Buffer pH                          ',layout=label_layout),pH_text]))
rows.append(ipw.HBox([ipw.Label('Equilibrium constant ($pK_1$)      ',layout=label_layout),pKeq1_text]))
rows.append(ipw.HBox([ipw.Label('Delta concentration                ',layout=label_layout),delta_text]))
rows.append(ipw.HBox([ipw.Label('Number of data point required      ',layout=label_layout),nPoints_text]))

rows.append(ipw.HBox([btn_calc1]))

rows.append(ipw.HBox([out_L]))
rows.append(ipw.HBox([out_P]))

ipw.VBox(rows)